In [92]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [3]:
filimo_df = pd.read_csv('./data2.csv')
filimo_df.head()

,Name,Actors,Score,About,Genre,Crew,Image,Voted,Type
0,عطش,"[' بهرام رادان', ' فریبرز عرب نیا', ' شهرام حق...",0.88,NaN,"['اکشن', 'ماجراجویی']","[' محمدحسین فرح بخش', ' عبدالله علیخانی', ' فر...",https://static.cdn.asset.filimo.com/flmt/mov_9...,761,فیلم
1,یکی از میان همه,"[' سروش صحت', ' رامین راستاد', ' آزاده صمدی', ...",0.69,نیلوفر و نرگس بدون اطلاع همسرانشان مجبور به سف...,"['اکشن', 'هیجان انگیز']",[' ابراهیم شیبانی'],https://static.cdn.asset.filimo.com/flmt/mov_1...,280,فیلم
2,جاده جنایت,"[' لعیا زنگنه', ' کاوه آرمیده', ' حشمت الله آر...",0.76,وقتی کتایون نگرانی و اضطراب بیش از حد شوهرش – ...,"['اکشن', 'هیجان انگیز']","[' جهانبخش ایمانی', ' ذبیح الله رحمانی', ' آرز...",https://static.cdn.asset.filimo.com/flmt/mov_7...,530,فیلم
3,ساوالان,"[' مجید مظفری', ' محمد برسوزیان', ' فردوس کاوی...",0.9,روستای ساوالان هرسال توسط گروهی راهزن غارت می ...,"['اکشن', 'ماجراجویی']","[' یدالله صمدی', ' سیفعلی سیدکوثری نژاد', ' ید...",https://static.cdn.asset.filimo.com/flmt/mov_6...,493,فیلم
4,پوزه چرمی,"[' ندا مقصودی', ' وحید جباری']",0.68,مردی برای رسیدن به اهدافش با پس انداز همسرش دس...,"['فیلم تئاتر', 'خانوادگی']","[' محمدعلی زمانی', ' هلموت کراوزر', ' محمد مهد...",https://static.cdn.asset.filimo.com/flmt/mov_2...,182,فیلم


In [8]:
filimo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    1410 non-null   object
 1   Actors  1410 non-null   object
 2   Score   1410 non-null   object
 3   About   888 non-null    object
 4   Genre   1410 non-null   object
 5   Crew    1410 non-null   object
 6   Image   1410 non-null   object
 7   Voted   1408 non-null   object
 8   Type    1410 non-null   object
dtypes: object(9)
memory usage: 99.3+ KB


In [9]:
filimo_df.shape

(1410, 9)

In [10]:
features = ['Actors','Genre','Crew']
for f in features:
    filimo_df[f] = filimo_df[f].map(literal_eval)

In [14]:
filimo_df.loc[0,'Crew'][0]

' محمدحسین فرح بخش'

In [20]:
for f in features:
    max_len = filimo_df[f].apply(len)
    print(max(max_len))

40
2
20


In [30]:
def get_director(x):
    if len(x):
        return x[0].replace(' ','')
    return ''

In [51]:
def get_names(x):
    if len(x):
        if len(x) > 3:
            return [i.replace(' ', '') for i in x[:3]]
        else:
            return [i.replace(' ', '') for i in x]
    return []

In [156]:
filimo_df = filimo_df[filimo_df['Type']=='فیلم']
filimo_df.shape
filimo_df.reset_index(inplace=True)

In [157]:
filimo_df['director'] = filimo_df['Crew'].apply(get_director)

In [158]:
features = ['Actors','Genre']
for f in features:
    filimo_df[f] = filimo_df[f].apply(get_names)

In [182]:
def create_soup(x):
    return ' '.join(x['Actors']) + ' ' +  ' '.join(x['Genre']) + ' ' + x['director'] + ' ' + x['director']

In [183]:
filimo_df['soup'] = filimo_df.apply(create_soup,axis=1)

In [184]:
filimo_df.loc[0,'soup']

'بهرامرادان فریبرزعربنیا شهرامحقیقتدوست اکشن ماجراجویی محمدحسینفرحبخش محمدحسینفرحبخش'

In [185]:
cv = CountVectorizer()
soup_cv = cv.fit_transform(filimo_df['soup'])
cv.get_feature_names_out()

array(['hosseinnazer', 'ramtinlavafi', 'sahraakarimi', ..., 'یوسفمرادیان',
       'یوناتدین', 'یکتاناصر'], dtype=object)

In [186]:
soup_cv.shape

(1327, 1901)

In [187]:
example_df = pd.DataFrame({'word':cv.get_feature_names_out(), 'count': soup_cv[0].T.todense().tolist()})
example_df.sort_values(by='count', ascending=False).head(10)

,word,count
1308,محمدحسینفرحبخش,[2]
894,شهرامحقیقتدوست,[1]
1170,فریبرزعربنیا,[1]
1217,ماجراجویی,[1]
280,بهرامرادان,[1]
238,اکشن,[1]
1420,مرجانهگلچین,[0]
1261,مجیدیاسر,[0]
1273,محسنزارعیان,[0]
1272,محسنرمضانی,[0]


In [188]:
cosine_sim = cosine_similarity(soup_cv, soup_cv)

In [189]:
cosine_sim.shape

(1327, 1327)

In [190]:
filimo_df.index.values

array([   0,    1,    2, ..., 1324, 1325, 1326])

In [191]:
titles = pd.Series(filimo_df.index.values, index=filimo_df['Name'])
titles.tail()

Name
 حمال طلا              1322
 بهشت پنهان            1323
 جدا افتاده            1324
 پنجمین سوار سرنوشت    1325
 بید مجنون             1326
dtype: int64

In [194]:
def get_recommendations(x, cosine_sim = cosine_sim):
    n = cosine_sim.shape[0]
    indices = titles[x]
    x_cos_sim = cosine_sim[indices]
    indices_sim = (-x_cos_sim).argsort()[:n]
    titles_sim = []
    for i in range(1,11):
        titles_sim.append(titles.index[indices_sim[i]])
    return titles_sim

In [195]:
get_recommendations(' از کرخه تا راین')

[' دیده بان',
 ' برج مینو',
 ' به وقت شام',
 ' مهاجر',
 ' بادیگارد',
 ' به رنگ ارغوان',
 ' ارتفاع پست',
 ' به نام پدر',
 ' دعوت',
 ' خروج']

In [198]:
get_recommendations(' من ترانه 15 سال دارم')

[' دیشب باباتو دیدم آیدا',
 ' سال دوم دانشکده من',
 ' دختری با کفش های کتانی',
 ' شب',
 ' کنعان',
 ' انتهای خیابان هشتم',
 ' آستیگمات',
 ' رسم عاشق کشی',
 ' دو',
 ' دریا موج کاکا']